## 가사 전처리기

### 0. 라이브러리 불러오기

In [2]:
import pandas as pd
import numpy as np
from tqdm.notebook import tqdm
import time
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM
import nltk


### 1. 가사 전처리 방법

### 1. 전처리 방법 선택을 위한 라벨링 (정답)

In [42]:
music= pd.read_csv('music.csv')

In [43]:
music

,제목,가수,발매일,장르,가사,이미지
0,날 사랑했다면,박학기,1993.10.01,발라드,떠나버린 널 깨닫기엔\n하늘이 너무 맑아\n우리에게 필요한 건 시간일뿐\n\n영화속...,https://cdnimg.melon.co.kr/cm/album/images/005...
1,그대 떠난 이밤에,옥슨 80,1981.09.25,록/메탈,바람부네 아픈 내맘에\n아름다운 추억 못잊어 부네\n그대 떠난 이밤\n그대 떠난 이...,https://cdnimg.melon.co.kr/cm/album/images/000...
2,나는 홀로 있어도,유심초,1980.11.01,성인가요/트로트,하얀 불빛 아래에\n침묵만이 흐르고\n낯설은 네 눈길에\n눈물만이 흐르네\n멀어져 ...,https://cdnimg.melon.co.kr/cm/album/images/003...
3,Love Is (Dance Mega Mix Ver.),터보,2001.06.20,댄스,우린 6년전에 만났지\n\n널 사랑하게 됐어\n\n내 마음을 숨긴체\n\n널 따라 ...,https://cdnimg.melon.co.kr/cm/album/images/003...
4,사랑밖엔 난 몰라 (Original Ver.),심수봉,2009.08.13,성인가요/트로트,그대 내 곁에선 순간\n\n그 눈빛이 너무 좋아\n\n어제는 울었지만\n\n오늘은 ...,https://cdnimg.melon.co.kr/cm/album/images/006...
...,...,...,...,...,...,...
4406,작은 기다림,쿨 (COOL),1995.10.01,댄스,널 이렇게 보내 줄 수\n밖에 없었어\n나 후회할 지도 모른 채\n끝없이 다가오는 ...,https://cdnimg.melon.co.kr/cm/album/images/000...
4407,WE LIKE 2 PARTY,BIGBANG (빅뱅),2015.06.01,랩/힙합,오늘도 친구들이 왔어\nMAN HOW YOU BEEN WHATS UP\nAYE 여기...,https://cdnimg.melon.co.kr/cm/album/images/023...
4408,내 손을 잡아,아이유,2011.05.25,"발라드, 국내드라마",느낌이 오잖아\n\n떨리고 있잖아\n\n언제까지 눈치만 볼 거니\n\n네 맘을 말해...,https://cdnimg.melon.co.kr/cm/album/images/012...
4409,Lie (Radio Ver.),박화요비,2000.06.14,R&B/Soul,말해줄래 나를 위해라면\n이러진 말아\n아직 그댈 놓아주기에는\n너무 난 두려워\n...,https://cdnimg.melon.co.kr/cm/album/images/000...


In [46]:
music.columns

Index(['제목', '가수', '발매일', '장르', '가사', '이미지'], dtype='object')

### 2. 전처리 방법

#### 2-0) 줄바꿈 생략 및 특수문자 제거

In [36]:
def remove_newlines_and_special_chars(paragraph):
    paragraph = paragraph.replace("\n", " ")

    chars_to_remove = "!@#$%^&*()_+-=[]{};':\"<>,.?/\\|"

    return paragraph.translate(str.maketrans("", "", chars_to_remove))

In [49]:
music['가사'][0]

'떠나버린 널 깨닫기엔\n하늘이 너무 맑아\n우리에게 필요한 건 시간일뿐\n\n영화속의 사랑 얘기도\n힘든 순간이 있어\n우린 다시 만날거야\n영화속의 주인공처럼\n네가 택한 길이\n언제나 너의 건 아니야\n아직은 아무런 말도 하지 말아줘\n날 사랑했다면\n\n멀어지는 널 깨닫기엔\n하늘이 너무 맑아\n그대에게 필요한 건 시간일뿐\n\n영화속의 사랑 얘기도\n힘든 순간이 있어\n우린 다시 웃을거야\n영화속의 주인공처럼\n네가 택한 길이\n언제나 너의 건 아니야\n아직은 아무런 말도 하지 말아줘\n날 다시 한번만 바라봐\n나에게 다가올 시간이\n날 더욱더 슬퍼지게만 해\n예전의 밝은 미소를 내게 보여줘\n날 사랑했다면'

In [50]:
for i in range(len(music['가사'])):
     music['가사'][i] = remove_newlines_and_special_chars(music['가사'][i])

In [51]:
music['가사'][0]

'떠나버린 널 깨닫기엔 하늘이 너무 맑아 우리에게 필요한 건 시간일뿐  영화속의 사랑 얘기도 힘든 순간이 있어 우린 다시 만날거야 영화속의 주인공처럼 네가 택한 길이 언제나 너의 건 아니야 아직은 아무런 말도 하지 말아줘 날 사랑했다면  멀어지는 널 깨닫기엔 하늘이 너무 맑아 그대에게 필요한 건 시간일뿐  영화속의 사랑 얘기도 힘든 순간이 있어 우린 다시 웃을거야 영화속의 주인공처럼 네가 택한 길이 언제나 너의 건 아니야 아직은 아무런 말도 하지 말아줘 날 다시 한번만 바라봐 나에게 다가올 시간이 날 더욱더 슬퍼지게만 해 예전의 밝은 미소를 내게 보여줘 날 사랑했다면'

#### 2-1) Pororo (가사 축약) in Colab

In [ ]:
!pip install konlpy

In [ ]:
!pip install git+https://github.com/kakaobrain/pororo.git

In [ ]:
!pip install python-mecab-ko

In [ ]:
from tqdm import tqdm
from pororo import Pororo

In [ ]:
# pororo 라이브러리 사용 model은 "abstractive"
abs_summ = Pororo(task="text_summarization", lang="ko", model="abstractive")

In [4]:
sample_music.columns

Index(['제목', '가수', '발매일', '장르', '가사', '이미지'], dtype='object')

In [ ]:
sample_music['pororo_pre'] = np.nan

In [ ]:
for i in tqdm(range(len(sample_music['가사']))):
      #매개 변수들은 default값과 전부 동일
  lyric = sample_music['가사'][i]
  sample_music['pororo_pre'][i] = abs_summ(
      lyric,
      beam=5,
      len_penalty=0.6,
      no_repeat_ngram_size=3,
      top_k=50,
      top_p=0.7
  )

In [ ]:
sample_music.isnull().sum()

In [ ]:
sample_music.loc[0] #pororo_pre column에 축약된 가사 확인

#### 2-2) textrank

In [ ]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM
import nltk
import pandas as pd
import numpy as np
nltk.download('punkt')
model_dir = "lcw99/t5-base-korean-text-summary"
tokenizer = AutoTokenizer.from_pretrained(model_dir)
model = AutoModelForSeq2SeqLM.from_pretrained(model_dir)

max_input_length = 512

In [ ]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM
import nltk
sample_music['요약가사'] = np.nan
nltk.download('punkt')

model_dir = "lcw99/t5-base-korean-text-summary"
tokenizer = AutoTokenizer.from_pretrained(model_dir)
model = AutoModelForSeq2SeqLM.from_pretrained(model_dir)

max_input_length = 512
for i in tqdm(range(len(sample_music['전처리가사']))):

    text = sample_music['전처리가사'][i]

    inputs = ["summarize: " + text]

    inputs = tokenizer(inputs, max_length=max_input_length, truncation=True, return_tensors="pt")
    output = model.generate(**inputs, num_beams=8, do_sample=True, min_length=10, max_length=100)
    decoded_output = tokenizer.batch_decode(output, skip_special_tokens=True)[0]
    predicted_title = nltk.sent_tokenize(decoded_output.strip())[0]

    sample_music['요약가사'][i] =predicted_title

#### 2-3) half (가사 축약)

In [ ]:
sample_music['half_pre'] = np.nan

In [ ]:
sample_music[0]

In [ ]:
len(sample_music['가사'][0])

In [ ]:
sample_music['가사'][0][ :len(sample_music['가사'][0]) // 2]

In [ ]:
for i in range(len(sample_music['가사'])):
  sample_music['half_pre'][i] = sample_music['가사'][i][ :(len(sample_music['가사'][i]) // 2)]

In [ ]:
sample_music.isnull().sum()

#### 2-4) origin (가사 원문)

In [ ]:
sample_music.rename(columns = {'가사':'origin'}, inplace =True)

#### 2-5) augment (가사 확장) in Colab

##### 2-5-1) NLP_augment :"KoGPT" : Failed

In [ ]:
!pip install -q torch~=1.9.0

In [ ]:
!pip install -q transformers~=4.12.0

In [ ]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM

In [ ]:
tokenizer = AutoTokenizer.from_pretrained(
    'kakaobrain/kogpt', revision='KoGPT6B-ryan1.5b-float16',
    bos_token='[BOS]', eos_token='[EOS]', unk_token='[UNK]', pad_token='[PAD]', mask_token='[MASK]'
)

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print('Current device:', device)

In [ ]:
model = AutoModelForCausalLM.from_pretrained(
    'kakaobrain/kogpt', revision='KoGPT6B-ryan1.5b-float16',
    pad_token_id=tokenizer.eos_token_id,
    torch_dtype=torch.float16, low_cpu_mem_usage=True
).to(device=device, non_blocking=True)
_ = model.eval()

In [ ]:
def gpt(prompt, max_length: int = 256):
    with torch.no_grad():
    tokens = tokenizer.encode(prompt, return_tensors='pt').to(device=device, non_blocking=True)
    gen_tokens = model.generate(tokens, do_sample=True, temperature=0.8, max_length=max_length)
    generated = tokenizer.batch_decode(gen_tokens)[0]
    return generated

In [ ]:
gpt(prompt)

##### 2-5-2) NLP_augment :"EDA: Easy Data Augmentation Techniques for Boosting Performance on Text Classification Tasks" : Success

In [ ]:
!pip install -U nltk

In [ ]:
# EDA: Easy Data Augmentation Techniques for Boosting Performance on Text Classification Tasks clone
!git clone https://github.com/jasonwei20/eda_nlp.git

In [ ]:
# KorEDA clone
# wordnet은 KAIST에서 만든 Korean WordNet(KWN)을 사용함.
!git clone https://github.com/catSirup/KorEDA.git

In [ ]:
import random
import pickle
import re

wordnet = {}
with open("/content/KorEDA/wordnet.pickle", "rb") as f:
	wordnet = pickle.load(f)

In [ ]:
wordnet

In [ ]:
from eda import EDA

EDA("""숨을 크게 쉬어봐요
당신의 가슴 양쪽이 저리게
조금은 아파올 때까지
숨을 더 뱉어봐요
당신의 안에 남은 게 없다고
느껴질 때까지
숨이 벅차올라도 괜찮아요
아무도 그댈 탓하진 않아
가끔은 실수해도 돼
누구든 그랬으니까
괜찮다는 말
말뿐인 위로지만
누군가의 한숨
그 무거운 숨을
내가 어떻게
헤아릴 수가 있을까요
당신의 한숨
그 깊일 이해할 순 없겠지만
괜찮아요
내가 안아줄게요
숨이 벅차올라도 괜찮아요
아무도 그댈 탓하진 않아
가끔은 실수해도 돼
누구든 그랬으니까
괜찮다는 말
말뿐인 위로지만
누군가의 한숨
그 무거운 숨을
내가 어떻게
헤아릴 수가 있을까요
당신의 한숨
그 깊일 이해할 순 없겠지만
괜찮아요
내가 안아줄게요
남들 눈엔 힘 빠지는
한숨으로 보일진 몰라도
나는 알고 있죠
작은 한숨 내뱉기도 어려운 하루를 보냈단 걸
이제 다른 생각은 마요
깊이 숨을 쉬어봐요
그대로 내뱉어요
누군가의 한숨
그 무거운 숨을
내가 어떻게
헤아릴 수가 있을까요
당신의 한숨
그 깊일 이해할 순 없겠지만 괜찮아요
내가 안아줄게요
정말 수고했어요""",  alpha_sr=0.1, alpha_ri=0.1, alpha_rs=0.5, p_rd=0.5, num_aug=9)
# rd 와 rs만 사용 sr과 ri는 가사데이터 본연의 의미 왜곡의 문제

In [ ]:
for i in tqdm(range(len(sample_music['가사']))):
      #매개 변수들은 default값과 전부 동일
  lyric = sample_music['가사'][i]
  sample_music['aug_pre'][i] = EDA(
      alpha_sr=0.1, alpha_ri=0.1, alpha_rs=0.5, p_rd=0.5, num_aug=9
  )

### 3. 전처리 방법별 scoring 통합 (DataFrame형태)

#### 3-1) 전처리한 결과값과 기존 가사 전문 결과값 비교분석

In [7]:
total_pre = pd.read_csv('total_pre.csv')

In [8]:
total_score = pd.read_csv('total_score.csv')

In [31]:
total_score

,title,singer,index,origin,textrank_pre,half_pre,pororo_pre,aug_pre
0,가로수 그늘아래 서면,이문세,0,171,257.0,192.0,459.0,209.0
1,임과 함께,남진,97,168,523.0,132.0,428.0,202.0
2,달의 몰락,울랄라세션,20,191,553.0,515.0,374.0,455.0
3,사랑 먼지,버블 시스터즈,39,181,379.0,268.0,591.0,223.0
4,사랑은...향기를 남기고,테이,58,164,349.0,207.0,501.0,202.0
5,"1,2,3,4 (원,투,쓰리,포)",이하이,77,51,223.0,82.0,301.0,91.0
6,취향저격,iKON,114,84,264.0,86.0,198.0,120.0
7,Touch My Body,씨스타,134,78,318.0,94.0,183.0,97.0


### 4. 최종 전처리 과정

In [33]:
music = pd.read_csv('music.csv')

In [52]:
for i in range(len(music['가사'])):
     music['가사'][i] = remove_newlines_and_special_chars(music['가사'][i])

In [40]:
music

,제목,가수,발매일,장르,가사,이미지
0,날 사랑했다면,박학기,1993.10.01,발라드,떠나버린 널 깨닫기엔 하늘이 너무 맑아 우리에게 필요한 건 시간일뿐 영화속의 사랑...,https://cdnimg.melon.co.kr/cm/album/images/005...
1,그대 떠난 이밤에,옥슨 80,1981.09.25,록/메탈,바람부네 아픈 내맘에 아름다운 추억 못잊어 부네 그대 떠난 이밤 그대 떠난 이밤이 ...,https://cdnimg.melon.co.kr/cm/album/images/000...
2,나는 홀로 있어도,유심초,1980.11.01,성인가요/트로트,하얀 불빛 아래에 침묵만이 흐르고 낯설은 네 눈길에 눈물만이 흐르네 멀어져 간 발길...,https://cdnimg.melon.co.kr/cm/album/images/003...
3,Love Is (Dance Mega Mix Ver.),터보,2001.06.20,댄스,우린 6년전에 만났지 널 사랑하게 됐어 내 마음을 숨긴체 널 따라 다녀었지 ...,https://cdnimg.melon.co.kr/cm/album/images/003...
4,사랑밖엔 난 몰라 (Original Ver.),심수봉,2009.08.13,성인가요/트로트,그대 내 곁에선 순간 그 눈빛이 너무 좋아 어제는 울었지만 오늘은 당신 땜에 ...,https://cdnimg.melon.co.kr/cm/album/images/006...
...,...,...,...,...,...,...
4406,작은 기다림,쿨 (COOL),1995.10.01,댄스,널 이렇게 보내 줄 수 밖에 없었어 나 후회할 지도 모른 채 끝없이 다가오는 너와의...,https://cdnimg.melon.co.kr/cm/album/images/000...
4407,WE LIKE 2 PARTY,BIGBANG (빅뱅),2015.06.01,랩/힙합,오늘도 친구들이 왔어 MAN HOW YOU BEEN WHATS UP AYE 여기 한...,https://cdnimg.melon.co.kr/cm/album/images/023...
4408,내 손을 잡아,아이유,2011.05.25,"발라드, 국내드라마",느낌이 오잖아 떨리고 있잖아 언제까지 눈치만 볼 거니 네 맘을 말해봐 딴청 ...,https://cdnimg.melon.co.kr/cm/album/images/012...
4409,Lie (Radio Ver.),박화요비,2000.06.14,R&B/Soul,말해줄래 나를 위해라면 이러진 말아 아직 그댈 놓아주기에는 너무 난 두려워 이미 지...,https://cdnimg.melon.co.kr/cm/album/images/000...


In [39]:
music['가사'][1]

'바람부네 아픈 내맘에 아름다운 추억 못잊어 부네 그대 떠난 이밤 그대 떠난 이밤이 깊어갈수록 사랑의 노래 울려퍼지네 우리 사랑에 오 해가저문 이밤에 다시 불러보네 아름다운 추억의 사랑의 노래 이제 하염없이 흐르는 눈물자욱 위에 떠가네 그대 떠난 이밤에 다시 불러보네 외로워진 마음의 사랑의 노래 나는 그대의 사랑노래 고이 간직하며 살리라 비가 오면 아름다워라 사랑의 추억 아름다워라 그대 떠난 이밤에 그대 떠난 이밤이 깊어갈수록 사랑노래 울려퍼지네 우리 사랑이 울려퍼지네 오 해가 저문 이밤에 다시 불러보네 아름다운 추억의 사랑의 노래 이제 하염없이 흐르는 눈물자욱 위에 떠가네 그대 떠난 이밤에 다시 불러보네 외로워진 마음의 사랑의 노래 나는 그대의 사랑노래 고이 간직하며 살리라 눈 내리는 창가에 앉아 포근한 추억 어루만지네 그대 떠난 이밤 그대 떠난 이밤이 깊어갈수록 사랑노래 울려퍼지네 우리 사랑에 해가 저문 이밤에 다시 불러보네 아름다운 추억의 사랑의 노래 이제 하염없이 흐르는 눈물자욱 위에 떠가네 그대 떠난 이밤에 다시 불러보네 외로워진 마음의 사랑의 노래 나는 그대의 사랑노래 고이 간직하며 살리라 나는 그대의 사랑노래 고이 간직하며 살리라'

In [53]:
music.to_csv('music_pre.csv')